# Parsing data

In [79]:
import csv
import re

In [181]:
infile = 'ca01'
with open(infile,'rb') as f:
#     dialect=csv.Sniffer().sniff(f.read(1024),delimiters=" ,;")
    words = []
    tags = []
    reader=csv.reader(f,delimiter = "\n")
    for row in reader:
        if len(row)>0:
            words_tagged = row[0].split()
            words.extend([re.split(r'/',elem)[0] for elem in words_tagged])
            tags.extend([re.split(r'/',elem)[1] for elem in words_tagged])
    print len(words),len(tags)
    dict = sorted(set(words))

2242 2242


# Preprocessing text

In [235]:
import os
import nltk
import re
import numpy
from nltk.tokenize import sent_tokenize
# from nltk.corpus import stopwords
# from nltk.stem.porter import *
# from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

data_path = "/home/debojyoti/brown/"
files=[os.path.join(data_path,f) for f in os.listdir(data_path) if re.match(r'c.+',f)!=None]
vocab = set([])
# stemmer = PorterStemmer()
# stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()
print wordnet_lemmatizer.lemmatize('said',pos='v')
files=["ca01"]
for infile in files:
    with open(infile) as myfile:
        data_orig=myfile.read()
        data = re.sub(r"(\S+)/(\S+)",r"\1",data_orig,flags=re.DOTALL)
        labels = re.sub(r"(\S+)/(\S+)",r"\2",data_orig,flags=re.DOTALL)
        # whitespace removal and Stemming
#         data = " ".join(data.split())
#         data = " ".join([stemmer.stem(elem) for elem in data.split()])
        lst=[]
        for elem,label in zip(data.split(),labels.split()):
            if re.match(r'vb.*',label)!=None:
                lst.append(wordnet_lemmatizer.lemmatize(elem,pos='v'))
            elif re.match(r'nn.*',label)!=None:
                lst.append(wordnet_lemmatizer.lemmatize(elem,pos='n'))
            else:
                lst.append(elem)
        lst=[str(i) for i in lst]
#         print lst
        data = " ".join(lst)
#         print data
        # Replace number with pseudowords
        data = re.sub(r"\d+(.\d+)?","NUM",data)
        # Tokenize
        tokens = nltk.word_tokenize(data)
        # Build vocabulary
        vocab.update(set(tokens))
vocabulary = numpy.asarray(sorted(vocab))
indices = numpy.arange(vocabulary.size)
word2indx = {}
for key,val in zip(vocabulary,indices):
    word2indx[key]=val
sentences = sent_tokenize(data)

say


# Creating Softmax Model

In [3]:
import theano,numpy
import theano.tensor as T

In [32]:
n_in = 10
n_out = 5
n_samples = 12
# Initialize weight matrix
W = theano.shared(
    value = numpy.ones(
        (n_in,n_out),
        dtype = theano.config.floatX
    ),
    name = 'W',
    borrow = True
)
# print W.eval()
# Initialize bias
b = theano.shared(
    value = numpy.ones(
        (n_out,),
        dtype = theano.config.floatX
    ),
    name = 'b',
    borrow = True
)
X=numpy.random.rand(n_samples,n_in)
p_y_given_x = T.nnet.softmax(T.dot(X,W) + b)
# p_y_given_x.eval()
y_pred = T.argmax(p_y_given_x, axis=1)

## Loss function: Negative Log likelihood

In [40]:
y=numpy.random.randint(5,size=n_samples)
-T.mean(T.log(p_y_given_x)[T.arange(y.size),y]).eval()

1.6094379124340998

# Logistic Regression

In [62]:
class LogisticRegression(object):
    def __init__(self, inpt, n_in, n_out):
        self.W = theano.shared(
            value = numpy.zeros(
                (n_in,n_out),
                dtype = theano.config.floatX
            ),
            name = 'W',
            borrow = True
        )
        self.b = theano.shared(
            value = numpy.zeros(
                (n_out,),
                dtype = theano.config.floatX
            ),
            name = 'b',
            borrow = True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(inpt,self.W) + self.b)
        self.y_pred = T.argmax(p_y_given_x, axis=1)
        self.params = [self.W,self.b]
        self.inpt = inpt
    
    def negative_log_likelihood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.size),y])

In [63]:
# Generate symbolic variables for input and labels
x = T.matrix('x')
y = T.ivector('y')
classifier = LogisticRegression(inpt=x, n_in=20*20, n_out=10)

In [64]:
# handle for cost
cost = classifier.negative_log_likelihood(y)

In [66]:
# Gradients w.r.t W and b
g_W=T.grad(cost=cost,wrt=classifier.W)
g_b=T.grad(cost=cost,wrt=classifier.b)

In [69]:
# Update Eqn
learning_rate=T.dscalar('learning_rate')
updates = [(classifier.W,classifier.W-learning_rate*g_W),
           (classifier.b,classifier.b-learning_rate*g_b)]

In [4]:
# Training Model
# index=T.iscalar('index')
# train_set_x=T.matrix('train_set')
# train_set_y=T.dvector('labels')
# batch_size=T.iscalar('batch_size')
# train_model = theano.function(
#     inputs=[index],
#     outputs=cost,
#     updates=updates,
#     givens={
#         x: train_set_x[index * batch_size: (index + 1) * batch_size],
#         y: train_set_y[index * batch_size: (index + 1) * batch_size]
#     }
# )

## Generator object

In [242]:
mygenerator = (x*x for x in range(3))
for i in mygenerator:
    print i

0
1
4


In [254]:
def example():
    for i in range(5):
        yield i
ex=example()
x=next(ex)
y=next(ex)
print x,y

0 1
